In [1]:
System.out.println("Hello World!")

Hello World!


In [ ]:
%%jars
app/target/lib

Found 151 jar files.
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/spring-beans-6.2.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/spring-boot-starter-3.4.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/jakarta.activation-api-2.1.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/snakeyaml-2.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/spring-grpc-test-0.11.0-SNAPSHOT.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/accessors-smart-2.5.2.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/junit-4.13.2.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/spring-grpc-server-spring-boot-starter-0.11.0-SNAPSHOT.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/objenesis-3.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/lib/spring-grpc-spring-boot-starter-0.11.0-SNAPSHOT.jar to classpath
Add /home/dsyer/

In [ ]:
%classpath app/target/test-classes/
%classpath app/target/classes/

Add /home/dsyer/dev/scratch/grpc-reflect/target/test-classes to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/target/classes to classpath


In [4]:
import com.google.protobuf.DescriptorProtos.FileDescriptorProto;
import com.google.protobuf.DescriptorProtos.DescriptorProto;
import com.google.protobuf.DescriptorProtos.FieldDescriptorProto;
import com.google.protobuf.DescriptorProtos.ServiceDescriptorProto;
import com.google.protobuf.DescriptorProtos.MethodDescriptorProto;

In [5]:
import com.google.protobuf.Descriptors.FileDescriptor;
import com.google.protobuf.Descriptors.Descriptor;
import com.google.protobuf.Descriptors.FieldDescriptor;

In [6]:
import com.google.protobuf.GeneratedMessage;
import com.google.protobuf.DynamicMessage;

To open up reflection for the fields in a message you need a `FileDescriptor`. The file descriptor has a list of message types that are themselves each a `Descriptor`. This is the `Descriptor` for the `Foo` message:


In [7]:
import com.google.protobuf.*;

In [40]:
var echo = ServiceDescriptorProto.newBuilder()
	.setName("Foo")
	.addMethod(
		MethodDescriptorProto.newBuilder()
			.setName("Echo")
			.setInputType("FooRequest")
			.setOutputType("FooRequest")
			.build()
	).build();

In [41]:
var foot = DescriptorProto.newBuilder()
.setName("FooRequest")
.addField(
	FieldDescriptorProto.newBuilder()
		.setName("name")
		.setNumber(1)
		.setType(FieldDescriptorProto.Type.TYPE_STRING)
)
.addField(
	FieldDescriptorProto.newBuilder()
		.setName("age")
		.setNumber(2)
		.setType(FieldDescriptorProto.Type.TYPE_INT32)
).build();

In [42]:
var food = FileDescriptorProto.newBuilder()
	.setName("foo.proto")
	.setSyntax("proto3")
	.addMessageType(foot)
	.addService(echo)
	.build()

In [43]:
var desc = FileDescriptor.buildFrom(food, new FileDescriptor[0]).getMessageTypes().get(0)

In [44]:
desc.getName()

FooRequest

In [45]:
var service = FileDescriptor.buildFrom(food, new FileDescriptor[0]).getServices().get(0)

In [31]:
desc.getFields()

[Foo.name, Foo.age]

Now `desc` is the "pure" form of the `DescriptorProto` called `foot`. A `Descriptor` can be inspected to pull out field metadata:

In [141]:
desc.findFieldByName("name").toProto()

name: "name"
number: 1
type: TYPE_STRING


In [46]:
var foo = DynamicMessage.newBuilder(desc).setField(
	desc.findFieldByName("name"), "foo"
).build()

In [16]:
import io.grpc.MethodDescriptor;
import io.grpc.protobuf.ProtoUtils;

In [47]:
service.getFullName()

Foo

In [48]:
var marshaller = ProtoUtils.marshaller(DynamicMessage.newBuilder(foot).build());
var methodDescriptor = MethodDescriptor
	.<DynamicMessage, DynamicMessage>newBuilder()
	.setType(MethodDescriptor.MethodType.UNKNOWN)
	.setFullMethodName(
			MethodDescriptor.generateFullMethodName(service.getFullName(), "Echo"))
	.setRequestMarshaller(marshaller)
	.setResponseMarshaller(marshaller)
	.build();

In [ ]:
import io.grpc.netty.NettyChannelBuilder;
import io.grpc.InsecureChannelCredentials;
import io.grpc.CallOptions;


In [33]:
import io.grpc.ClientCall;
import io.grpc.stub.ClientCalls;
import io.grpc.Metadata;

In [62]:
var channel = NettyChannelBuilder.forTarget("localhost:9090", InsecureChannelCredentials.create()).build();

In [63]:
var response = ClientCalls.blockingUnaryCall(channel, methodDescriptor, CallOptions.DEFAULT, foo)

In [58]:
response

name: "foo"


Alternative form:

In [ ]:
var call = channel.newCall(methodDescriptor, CallOptions.DEFAULT);
var response = ClientCalls.blockingUnaryCall(call, foo)

name: "foo"
